<a href="https://colab.research.google.com/github/SamMukhPro/MCAProjects/blob/main/Major_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**FILE UPLOAD**

In [ ]:
from google.colab import files

uploaded = files.upload()


filename = list(uploaded.keys())[0]


with open(filename, "r", encoding="utf-8-sig") as f:
    text = f.read()

print(text[:500])


Saving Untitled document.txt to Untitled document.txt
তরুণের বিদ্রোহবন্ধুগণ,নিজের জীবন এলো যখন সমাপ্তির দিকে, তখন ডাক পড়লো আমার দেশের এই যৌবন-শক্তিকে সম্বোধন ক’রে তাদের যাত্রাপথের সন্ধান দিতে। নিজের মধ্যে কর্মশক্তি যখন নিঃশেষিতপ্রায়, উদ্যম ক্লান্ত, প্রেরণা ক্ষীণ, তখন তরুণের অপরিমেয় প্রাণধারার দিক-নির্ণয়ের ভার পড়লো এক বৃদ্ধের উপর। এ আহ্বানে সাড়া দিবার শক্তি-সামর্থ্য নেই—সময় গেছে। এ আহানে বুকের মধ্যে শুধু বেদনার সঞ্চার করে।


**(B) LANGUAGE-SPECIFIC FEATURES**

In [ ]:
import re
def extract_juktakkhor_features(text,filename):
    # Regex for Bengali Consonant range
    # \u0985 to \u09B9 covers the main Bengali alphabet
    consonant = r'[\u0985-\u09B9]'
    hasanta = r'\u09CD'

    # Detect 2+ consonant conjunct clusters
    juktakkhor_pattern = f'{consonant}(?:{hasanta}{consonant})+'

    # 3. Find all true Juktakkhors
    all_juks = re.findall(juktakkhor_pattern, text)
    total_juks = len(all_juks)

    # 4. Count sentences (splitting by Dari, Question mark, or Exclamation)
    sentences = re.split(r'[।!?]', text)
    # Filter out empty strings from the list
    sentences = [s for s in sentences if s.strip()]
    num_sentences = len(sentences)

    # 5. Calculate AJuk (Total) and JUK (Average)
    ajuk = total_juks
    juk_per_sentence = total_juks / num_sentences if num_sentences > 0 else 0

    return {
        "Filename": filename,
        "Total_Juktakkhor (AJuk)": ajuk,
        "Avg_per_Sentence (JUK)": round(juk_per_sentence, 3),
        "Samples_Found": all_juks  # Shows the first 10 for verification
    }

# --- Execution ---
print("\n--- Processing Results ---")
results = extract_juktakkhor_features(text,filename)
for key, value in results.items():
    print(f"{key}: {value}")


--- Processing Results ---
Filename: Untitled document.txt
Total_Juktakkhor (AJuk): 24
Avg_per_Sentence (JUK): 6.0
Samples_Found: ['দ্র', 'ন্ধ', 'প্ত', 'ক্ত', 'ম্ব', 'ত্র', 'ন্ধ', 'ধ্য', 'র্ম', 'ক্ত', 'প্র', 'দ্য', 'ক্ল', 'ন্ত', 'প্র', 'ক্ষ', 'প্র', 'র্ণ', 'দ্ধ', 'হ্ব', 'ক্ত', 'র্থ্য', 'ধ্য', 'ঞ্চ']


In [ ]:
pip install indic-nlp-library nltk


Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 28.69 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


AttributeError: module 'indicnlp.common' has no attribute 'get_stopwords'

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**SHALLOW/LEXICAL FEATURES**

In [ ]:
# ===============================================
# Bengali Readability – Shallow / Lexical Features
# ===============================================

from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import nltk
import re

# -----------------------------------------------
# Bengali linguistic resources
# -----------------------------------------------

BENGALI_VOWELS = set("অআইঈউঊঋএঐওঔািীুূৃেৈোৌ")

BENGALI_STOPWORDS = {
    "ও","এবং","যে","এই","তার","একটি","হয়","ছিল","করে","কে",
    "আমি","তুমি","সে","আমরা","তারা","এর","না","থেকে","মধ্যে"
}

BENGALI_PREFIXES = ("অ","অন","প্র","উপ","বি","নি","সম","পরি","অতি")
BENGALI_SUFFIXES = ("তা","ত্ব","কারী","করণ","ভাবে","গুলি","দের","দেরকে")

# -----------------------------------------------
# Helper functions
# -----------------------------------------------

def count_syllables(word):
    return sum(1 for ch in word if ch in BENGALI_VOWELS)

def count_affixes(word):
    count = 0
    for p in BENGALI_PREFIXES:
        if word.startswith(p) and len(word) > len(p):
            count += 1
    for s in BENGALI_SUFFIXES:
        if word.endswith(s) and len(word) > len(s):
            count += 1
    return count

# -----------------------------------------------
# MAIN FEATURE EXTRACTION
# -----------------------------------------------

def extract_bengali_features(text):

    # Normalize Bengali text
    normalizer = IndicNormalizerFactory().get_normalizer("bn")
    text = normalizer.normalize(text)

    # Sentence split
    sentences = nltk.sent_tokenize(text)
    num_sentences = len(sentences)

    # Word tokenize (Indic NLP)
    words = indic_tokenize.trivial_tokenize(text)

    # Keep Bengali words only
    words = [w for w in words if re.fullmatch(r"[\u0980-\u09FF]+", w)]

    total_words = len(words)
    if total_words == 0:
        return {}

    # Feature counters
    char_lengths = [len(w) for w in words]
    syllables = [count_syllables(w) for w in words]

    six_char_words = sum(1 for l in char_lengths if l >= 6)
    monosyllabic_words = sum(1 for s in syllables if s == 1)
    polysyllabic_words = sum(1 for s in syllables if s >= 3)

    total_syllables = sum(syllables)
    affix_count = sum(count_affixes(w) for w in words)
    stopword_count = sum(1 for w in words if w in BENGALI_STOPWORDS)

    # Averages
    avg_sentence_length = total_words / num_sentences
    avg_word_length = sum(char_lengths) / total_words
    avg_syllables_per_word = total_syllables / total_words
    polysyllables_per_sentence = polysyllabic_words / num_sentences

    return {
        "1. Word_Lengths_(characters)": char_lengths,
        "2. Words_>=6_Characters": six_char_words,
        "3. Total_Syllables": total_syllables,
        "4. Monosyllabic_Words": monosyllabic_words,
        "5. Words_>=3_Syllables": polysyllabic_words,
        "6. Number_of_Affixes": affix_count,
        "7. Average_Sentence_Length": round(avg_sentence_length, 3),
        "8. Average_Word_Length": round(avg_word_length, 3),
        "9. Polysyllabic_Words_per_Sentence": round(polysyllables_per_sentence, 3),
        "10. Average_Syllables_per_Word": round(avg_syllables_per_word, 3),
        "11. Stopwords_per_Document": stopword_count
    }

In [ ]:
features = extract_bengali_features(text)

print("\nBengali Shallow / Lexical Features:\n")
for k, v in features.items():
    print(f"{k}: {v}")


Bengali Shallow / Lexical Features:

1. Word_Lengths_(characters): [6, 14, 5, 4, 3, 3, 8, 4, 3, 3, 5, 4, 5, 2, 4, 7, 7, 1, 2, 5, 10, 6, 4, 5, 5, 9, 3, 14, 5, 7, 7, 5, 3, 6, 8, 10, 3, 9, 3, 5, 2, 7, 3, 1, 7, 5, 5, 5, 8, 3, 4, 4, 1, 5, 5, 5, 4, 6, 6, 3]
2. Words_>=6_Characters: 20
3. Total_Syllables: 92
4. Monosyllabic_Words: 24
5. Words_>=3_Syllables: 7
6. Number_of_Affixes: 13
7. Average_Sentence_Length: 60.0
8. Average_Word_Length: 5.183
9. Polysyllabic_Words_per_Sentence: 7.0
10. Average_Syllables_per_Word: 1.533
11. Stopwords_per_Document: 4


In [ ]:
!pip -q install ufal.udpipe indic-nlp-library


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 7.6 MB/s eta 0:00:00


In [ ]:
# Download from LINDAT/CLARIN official model repository
!wget -O /content/bengali-ud-2.17-251125.udpipe \
"https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-297C-0000-0029-0A34-2/bengali-ud-2.17-251125.udpipe?sequence=1&isAllowed=y"

# List to verify
!ls -lh /content/bengali-ud-2.17-251125.udpipe


--2026-02-16 15:49:39--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11858/00-297C-0000-0029-0A34-2/bengali-ud-2.17-251125.udpipe?sequence=1&isAllowed=y
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://lindat.mff.cuni.cz/repository/bitstream/handle/11858/00-297C-0000-0029-0A34-2/bengali-ud-2.17-251125.udpipe?sequence=1&isAllowed=y [following]
--2026-02-16 15:49:40--  https://lindat.mff.cuni.cz/repository/bitstream/handle/11858/00-297C-0000-0029-0A34-2/bengali-ud-2.17-251125.udpipe?sequence=1&isAllowed=y
Reusing existing connection to lindat.mff.cuni.cz:443.
HTTP request sent, awaiting response... 404 Not found
2026-02-16 15:49:41 ERROR 404: Not found.

-rw-r--r-- 1 root root 0 Feb 16 15:49 /content/bengali-ud-2.17-251125.udpipe


In [ ]:
import ufal.udpipe as udpipe
import os

MODEL_PATH = "/content/bengali-ud-2.17-251125.udpipe"

assert os.path.exists(MODEL_PATH), "❌ Model not found. Check the download cell."

def load_udpipe_pipeline():
    model = udpipe.Model.load(MODEL_PATH)
    if model is None:
        raise RuntimeError("Failed to load UDPipe model")

    pipeline = udpipe.Pipeline(
        model,
        "tokenize",
        "tag",
        "parse",
        "conllu"
    )
    print("✅ UDPipe Bengali pipeline ready")
    return pipeline

pipeline = load_udpipe_pipeline()



RuntimeError: Failed to load UDPipe model

In [ ]:
import ufal.udpipe as udpipe

model = udpipe.Model.load("bengali-bdt-ud-2.5-191206.udpipe")

pipeline = udpipe.Pipeline(
    model,
    "tokenize",
    udpipe.Pipeline.DEFAULT,
    udpipe.Pipeline.DEFAULT,
    "conllu"
)

print("✅ UDPipe Bengali pipeline ready")

pipeline = load_udpipe_pipeline()


✅ UDPipe Bengali pipeline ready


FileNotFoundError: UDPipe Bengali model not found at /content/bengali-bdt-ud-2.5-191206.udpipe

In [ ]:
import os

txt_files = [f for f in os.listdir('/content') if f.endswith('.txt')]
filename = '/content/' + txt_files[-1]

with open(filename, 'r', encoding='utf-8-sig') as f:
    text = f.read()

print("Loaded:", filename)


Loaded: /content/New Text Document (2) - Copy - Copy.txt


In [ ]:
def extract_bengali_pos_features(text):

    processed = pipeline.process(text)
    lines = processed.split("\n")

    counts = {
        "AJJ": 0,
        "ANN": 0,
        "ACC": 0,
        "PRON": 0,
        "ADV": 0,
        "PART": 0,
        "POS": 0
    }

    for line in lines:
        if line.startswith("#") or line.strip() == "":
            continue

        cols = line.split("\t")
        if len(cols) < 4:
            continue

        upos = cols[3]
        counts["POS"] += 1

        if upos == "ADJ":
            counts["AJJ"] += 1
        elif upos in ["NOUN", "PROPN"]:
            counts["ANN"] += 1
        elif upos in ["CCONJ", "SCONJ"]:
            counts["ACC"] += 1
        elif upos == "PRON":
            counts["PRON"] += 1
        elif upos == "ADV":
            counts["ADV"] += 1
        elif upos == "PART":
            counts["PART"] += 1

    return counts


In [ ]:
pos_features = extract_bengali_pos_features(text)

print("\nBengali POS Features:\n")
for k, v in pos_features.items():
    print(f"{k}: {v}")


NameError: name 'extract_bengali_pos_features' is not defined

In [ ]:
import os
import ufal.udpipe as udpipe

# Ensure pipeline is defined (re-initialize if necessary)
# This code is duplicated from eU2sUTZH3vMs to make this cell self-contained
if 'pipeline' not in locals() or 'pipeline' not in globals():
    try:
        model = udpipe.Model.load("bengali-bdt-ud-2.5-191206.udpipe")
        pipeline = udpipe.Pipeline(
            model,
            "tokenize",
            udpipe.Pipeline.DEFAULT,
            udpipe.Pipeline.DEFAULT,
            "conllu"
        )
        print("✅ UDPipe Bengali pipeline re-initialized for this cell")
    except Exception as e:
        print(f"Error initializing pipeline: {e}")
        pipeline = None # Set to None if initialization fails


# Ensure 'text' is defined by reloading the content from the last loaded file
txt_files = [f for f in os.listdir('/content') if f.endswith('.txt')]
if txt_files:
    filename_to_load = '/content/' + txt_files[-1]
    with open(filename_to_load, 'r', encoding='utf-8-sig') as f:
        text = f.read()
    print(f"Reloaded text from: {filename_to_load}")
else:
    print("No text files found in /content/ to reload.")

# Now, execute the feature extraction if pipeline is successfully initialized
if pipeline:
    pos_features = extract_bengali_pos_features(text)

    print("\nBengali POS Features:\n")
    for k, v in pos_features.items():
        print(f"{k}: {v}")
else:
    print("Cannot extract POS features: UDPipe pipeline failed to initialize.")

ModuleNotFoundError: No module named 'ufal'

In [ ]:
%%writefile bengali_pos_helper.py
import os
import ufal.udpipe as udpipe

MODEL_PATH = "bengali-bdt-ud-2.5-191206.udpipe"

def load_udpipe_pipeline():
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError("UDPipe Bengali model not found")

    model = udpipe.Model.load(MODEL_PATH)
    if model is None:
        raise RuntimeError("Model failed to load")

    return udpipe.Pipeline(
        model,
        "tokenize",
        "tag",
        "parse",
        "conllu"
    )

PIPELINE = load_udpipe_pipeline()

def extract_bengali_pos_features(text):
    processed = PIPELINE.process(text)
    lines = processed.split("\n")

    counts = {
        "AJJ": 0,
        "ANN": 0,
        "ACC": 0,
        "PRON": 0,
        "ADV": 0,
        "PART": 0,
        "POS": 0
    }

    for line in lines:
        if not line or line.startswith("#"):
            continue
        cols = line.split("\t")
        if len(cols) < 4:
            continue

        upos = cols[3]
        counts["POS"] += 1

        if upos == "ADJ":
            counts["AJJ"] += 1
        elif upos in ("NOUN", "PROPN"):
            counts["ANN"] += 1
        elif upos in ("CCONJ", "SCONJ"):
            counts["ACC"] += 1
        elif upos == "PRON":
            counts["PRON"] += 1
        elif upos == "ADV":
            counts["ADV"] += 1
        elif upos == "PART":
            counts["PART"] += 1

    return counts

Writing bengali_pos_helper.py


In [ ]:
from google.colab import files
files.download("bengali-bdt-ud-2.5-191206.udpipe")

In [ ]:
!pip -q install ufal.udpipe indic-nlp-library
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_Bengali-BDT/master/models/bengali-bdt-ud-2.5-191206.udpipe
print("✅ Libraries installed and model downloaded")


✅ Libraries installed and model downloaded


In [ ]:
import ufal.udpipe as udpipe
import os

MODEL_PATH = "/content/bengali-bdt-ud-2.5-191206.udpipe"



In [ ]:
def load_udpipe_pipeline():
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError("UDPipe Bengali model not found at " + MODEL_PATH)

    model = udpipe.Model.load(MODEL_PATH)
    if model is None:
        raise RuntimeError("Failed to load UDPipe model")

    pipeline = udpipe.Pipeline(
        model,
        "tokenize",
        "tag",
        "parse",
        "conllu"
    )
    print("✅ UDPipe Bengali pipeline ready")
    return pipeline

pipeline = load_udpipe_pipeline()


FileNotFoundError: UDPipe Bengali model not found at /content/bengali-bdt-ud-2.5-191206.udpipe

In [ ]:
!pip install bnltk


In [ ]:
from indicnlp.tokenize import indic_tokenize
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator
import os

print("✅ Indic NLP library ready")


✅ Indic NLP library ready


In [ ]:
txt_files = [f for f in os.listdir("/content") if f.endswith(".txt")]
if not txt_files:
    raise FileNotFoundError("💡 Upload a Bengali .txt file to /content first!")

filename = "/content/" + txt_files[-1]

with open(filename, "r", encoding="utf-8-sig") as f:
    text = f.read()

print(f"✅ Loaded text from: {filename[:60]}...")


✅ Loaded text from: /content/তরুণের বিদ্রোহ.txt...


In [ ]:
# You can expand these lists as needed
ADJECTIVES = ["ভালো", "মোটামুটি", "বড়", "ছোট", "নতুন"]  # common Bengali adjectives
NOUNS = ["ছেলে", "মেয়ে", "বই", "বিদ্যালয়", "শিক্ষক"]  # common nouns
CONJUNCTIONS = ["এবং", "কিন্তু", "অথবা", "যদি"]  # common conjunctions
PRONOUNS = ["আমি", "তুমি", "সে", "আমরা", "তাদের"]
ADVERBS = ["খুব", "ধীরে", "শিগগিরই", "ততক্ষণ"]
ARTICLES = ["একটি", "এই", "সেটি"]  # determiners / articles


In [ ]:
# Tokenize Bengali text
tokens = indic_tokenize.trivial_tokenize(text)

counts = {
    "ADJ": 0,
    "NOUN": 0,
    "CONJ": 0,
    "PRON": 0,
    "ADV": 0,
    "ART": 0,
    "POS_TOTAL": len(tokens)
}

for token in tokens:
    if token in ADJECTIVES:
        counts["ADJ"] += 1
    elif token in NOUNS:
        counts["NOUN"] += 1
    elif token in CONJUNCTIONS:
        counts["CONJ"] += 1
    elif token in PRONOUNS:
        counts["PRON"] += 1
    elif token in ADVERBS:
        counts["ADV"] += 1
    elif token in ARTICLES:
        counts["ART"] += 1


In [ ]:
print("\n📊 Bengali POS Features (approximation using Indic NLP):\n")
for k, v in counts.items():
    print(f"{k}: {v}")



📊 Bengali POS Features (approximation using Indic NLP):

ADJ: 12
NOUN: 2
CONJ: 60
PRON: 65
ADV: 2
ART: 45
POS_TOTAL: 3805


  Using cached bnlp_toolkit-4.4.0-py3-none-any.whl.metadata (14 kB)
  Using cached sentencepiece-0.2.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached gensim-4.3.2.tar.gz (23.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of bnlp-toolkit to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=b14efd57a783b8e49f776f531f8c8c61bea54600dec53e9799a

ImportError: cannot import name 'stopwords' from 'bnlp.corpus' (/usr/local/lib/python3.12/dist-packages/bnlp/corpus/__init__.py)

  Using cached stopwordsiso-0.6.1-py3-none-any.whl.metadata (2.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install stopwordsiso

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
import stopwordsiso as stopwords

bn_stopwords = list(stopwords.stopwords("bn"))
print("Total Bengali stopwords:", len(bn_stopwords))


print(*bn_stopwords, sep="\n")


Total Bengali stopwords: 398
অন্তত
সেটা
যাতে
তবু
দুই
আগেই
আমার
এতটাই
হয়
তাতে
হয়েছে
করিয়া
দিয়েছে
যাওয়া
কিছু
এটাই
পরেও
দেখে
উপরে
সেটি
অনেকেই
আদ্যভাগে
নেওয়া
হত
করতে
ফিরে
প্রাথমিক
ক্ষেত্রে
কেউই
বেশি
হয়ে
চেষ্টা
এই
যা
গেলে
তখন
কমনে
জানিয়েছে
কোনো
যখন
সেখান
স্পষ্ট
থেকেও
হোক
পারি
যাদের
এব
কোটি
ও
দেন
নেওয়ার
এখনও
এটি
বিশেষ
যার
তো
তারা
উপর
চলে
জন
বলেন
দুটি
হইয়া
সবার
অবধি
নতুন
থাকেন
এদের
মতো
কেন
গুলি
হয়েই
বদলে
কাছ
করেই
ওদের
ওখানে
করিতে
ওর
র
নয়
এঁরা
হলেও
জনের
সঙ্গেও
ইহা
বহু
অবশ্য
বলা
তিনি
যত
যাকে
এবং
নাই
কয়েকটি
তারৈ
ধরা
সহ
হতেই
তবে
তিনঐ
কবে
অতএব
হৈলে
রেখে
আগামী
এখানে
থাকায়
থাকা
গিয়ে
নিয়ে
নানা
থেকে
করার
যায়
কে
উত্তর
ছাড়াও
পারে
বা
কাছে
এর
বলেছেন
চার
এমনকী
করেন
হিসাবে
আপনি
পর্যন্ত
রাখা
এত
হয়েছেন
যারা
মোটেই
ওরা
সেই
থেকেই
যান
ধরে
করা
করলেন
কী
যাঁর
দেখা
প্রথম
অন্য
স্বয়ং
ফের
ছিলেন
কাউকে
পক্ষে
করে
নিয়ে
আজ
তাই
ওঁর
করি
এতে
পর
কাজে
পরেই
যাচ্ছে
কাজ
থাকবেন
চায়
বরং
পাচ
ওকে
করিয়ে
যেতে
ছাড়া
হন
হয়
তথা
হয়নি
সম্প্রতি
তিনিও
দিলেন
যদিও
জন্য
আমি
গোটা
অর্থাত
মধ্যেও
সাধারণ
জানায়
মধ্যে
তুলে
যেখানে
হল
হ